In [1]:
import numpy as np
import os
import glob
import matplotlib.image as mpimg
from PIL import Image

from resnet import batch_generator
from decomposer import Decomposer

from util.model_base import ModelBase
from util.helpers import *
from util.visualize import *
from util.cross_validation import cross_validate

Using TensorFlow backend.


In [2]:
image_dir = "training/images/"
gt_dir = "training/groundtruth/"

print("Loading training input... ", end = '')
X = [np.asarray(load_image((image_dir + file))) for file in os.listdir(image_dir)]
print("done (" + str(len(X)) + " images).")
X = np.array(X)

print("Loading training groundtruth... ", end = '')
Y = [np.asarray(load_image((gt_dir + file))) for file in os.listdir(gt_dir)]
print("done (" + str(len(Y)) + " images).")
Y = (np.array(Y) >= 0.21) * 1 # compensates for lossy image data

print()
print(X.shape)
print(Y.shape)

Loading training input... done (100 images).
Loading training groundtruth... done (100 images).

(100, 400, 400, 3)
(100, 400, 400)


In [3]:
class DecomposerTester(ModelBase):

    def __init__(self, bootstrap_path):
        self.bootstrap_path = bootstrap_path

    def train_online(self, generator):
        batch_gen = batch_generator(generator)
        
        X_batch, Z = next(batch_gen)
        Z = (Z[:, 0] < Z[:, 1]) * 1
        
        print("Created batch of shape "+ str(X_batch.shape) + " with groundtruth "+ str(Z.shape))
        
        files = glob.glob(r''+ self.bootstrap_path +'/*')
        for items in files:
            os.remove(items)
        
        for i in range(X.shape[0]):
            save_image(X_batch[i], self.bootstrap_path + "bs"+ str(i) +"_c"+ str(Z[i]) +".png")
        
    def classify(self, X):
        num_of_img = X.shape[0]
        img_size = (X.shape[1], X.shape[2])
        
        print("classifying X with shape = "+ str(X.shape))
        assert img_size == (72, 72) and X.shape[3] == 3, "Wrong shape during classification."

        Y = np.zeros(X.shape[0])
        print("returning Y with shape = "+ str(Y.shape))
        return Y

In [4]:
bootstrap_path = "training/bootstrap/"

model = Decomposer(DecomposerTester(bootstrap_path))

model.initialize()
model.train(Y, X)
Y_pred = model.classify(X)

Created batch of shape (250, 72, 72, 3) with groundtruth (250,)
classifying X with shape = (62500, 72, 72, 3)
returning Y with shape = (62500,)
